In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../src')


In [3]:
import pandas as pd

In [5]:
df = pd.read_parquet('../data/raw/raw_data.parquet')
df

,description,brand,original_price,offer_price
0,Aqua Shoes Sun Negro | Aquashoes Suela Gruesa ...,None,None,S/\n45
1,Zapatilla Skechers Uno Gen1-shimmer 155196/wht...,SKECHERS,None,S/\n279
2,Zapatillas Originales Hombre Nike Tiempo Legen...,NIKE,"S/\n289\n,\n90","S/\n173\n,\n94"
3,Aqua Shoes Sun Negro | Aquashoes Suela Gruesa ...,None,None,S/\n45
4,Zapatilla Skechers Uno Gen1-shimmer 155196/wht...,SKECHERS,None,S/\n279
...,...,...,...,...
46,Zapatillas Puma Hombre Caven 2.0 392290 35 Blanco,PUMA,None,None
47,Zapatillas Puma Mujer Skyrocket Lite Wn's 3101...,PUMA,None,None
48,Zapatillas adidas Hombre Urbanas Grand Court 2...,ADIDAS,S/\n279,S/\n182
49,Zapatos Castor Bubblegummers Para Niña,BUBBLEGUMMERS,"S/\n109\n,\n90","S/\n59\n,\n90"


In [7]:
from pipeline.steps.text_processing import TextProcessingStep
from pipeline.steps.cleaning import CleaningStep

df_cleaned = CleaningStep().execute(df)
df_preprocessed = TextProcessingStep().execute(df_cleaned)




In [8]:
df_preprocessed

,description,brand,original_price,offer_price,description_processed
0,aqua shoes sun negro aquashoes suela gruesa ...,None,NaN,45.00,aqua shoes sun negro aquashoes suela grueso + ...
1,zapatilla skechers uno gen1-shimmer 155196 wht...,SKECHERS,NaN,279.00,zapatillo skecher gen1-shimmer 155196 wht mujer
2,zapatillas originales hombre nike tiempo legen...,NIKE,289.9,173.94,zapatilla original hombre nike tiempo legend 1...
3,aqua shoes sun negro aquashoes suela gruesa ...,None,NaN,45.00,aqua shoes sun negro aquashoes suela grueso + ...
4,zapatilla skechers uno gen1-shimmer 155196 wht...,SKECHERS,NaN,279.00,zapatillo skecher gen1-shimmer 155196 wht mujer
...,...,...,...,...,...
43,zapatillas adidas urbanas grand court 2 0 k ...,ADIDAS,189.0,127.00,zapatilla adida urbano grand court 2 0 k id7870
44,zapatillas adidas unisex urbanas breaknet 2 0 ...,ADIDAS,179.0,121.00,zapatilla adida unisex urbano breaknet 2 0 ie3790
45,zapatilla puma caven 2 0 392291-04 para hombre,PUMA,345.0,248.40,zapatilla puma cavir 2 0 392291-04 hombre
48,zapatillas adidas hombre urbanas grand court 2...,ADIDAS,279.0,182.00,zapatilla adida hombre urbano grand court 2 0 ...


In [9]:
from pipeline.steps.feature_extraction import FeatureExtractionStep
from sklearn.model_selection import train_test_split

feature_extraction_step = FeatureExtractionStep(already_processed=True)
x_train, x_test, y_train, y_test = train_test_split(df_preprocessed[["description_processed"]], df_preprocessed["offer_price"], test_size=0.2, random_state=123)

x_train_vectorized = feature_extraction_step.execute(x_train, "description_processed", "train")
x_test_vectorized = feature_extraction_step.execute(x_test, "description_processed", "inference")


In [10]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.fit(x_train_vectorized, y_train)



model.predict(x_test_vectorized)


Learning rate set to 0.035425
0:	learn: 110.1177997	total: 58ms	remaining: 58s
1:	learn: 108.6984634	total: 58.7ms	remaining: 29.3s
2:	learn: 107.6397776	total: 59.4ms	remaining: 19.7s
3:	learn: 106.6456909	total: 59.9ms	remaining: 14.9s
4:	learn: 105.6314872	total: 60.4ms	remaining: 12s
5:	learn: 104.7281050	total: 61ms	remaining: 10.1s
6:	learn: 103.4097163	total: 61.5ms	remaining: 8.73s
7:	learn: 102.3265129	total: 62.1ms	remaining: 7.7s
8:	learn: 101.2947771	total: 62.6ms	remaining: 6.89s
9:	learn: 100.4803055	total: 63.2ms	remaining: 6.25s
10:	learn: 99.5114968	total: 63.7ms	remaining: 5.73s
11:	learn: 98.6272917	total: 64.3ms	remaining: 5.3s
12:	learn: 97.5502628	total: 64.9ms	remaining: 4.92s
13:	learn: 96.7099435	total: 65.4ms	remaining: 4.6s
14:	learn: 95.8865740	total: 65.9ms	remaining: 4.33s
15:	learn: 95.0172130	total: 66.4ms	remaining: 4.08s
16:	learn: 94.3083038	total: 66.9ms	remaining: 3.87s
17:	learn: 93.5857304	total: 67.4ms	remaining: 3.68s
18:	learn: 92.7283146	total

array([203.26314954, 167.28844906, 156.35910576,  67.62977824,
       142.77528105, 165.3110527 , 270.4717765 , 160.03082743,
       165.08228326, 154.12986888, 160.03082743,  93.99877135,
       158.96140056, 139.73027316, 209.82403599, 128.49078221,
       178.50660026, 218.77539233, 153.25740078,  99.7970904 ,
       140.13758954, 169.62517701, 229.70958719, 105.51355136,
       273.09862155, 134.93772469, 171.35988584, 126.17252295,
       209.82403599, 271.85093498,   2.49375307, 561.93473493,
       233.70917732, 446.16780364, 151.93629662, 156.35910576,
        67.62977824, 143.04198507, 144.82034616, 316.59540477,
       278.42380898, 181.30073974,  95.00956288, 167.33398383,
       165.08228326, 169.62517701, 108.87925851, 177.49044909,
       220.59005107, 241.94500881, 160.03082743, 224.62255026,
       210.39729904, 154.04482455, 156.12549584,  75.88115427,
       165.3110527 , 207.13230539, 241.94500881, 271.85093498,
       178.9886388 , 170.6996903 , 165.3110527 , 141.30

In [13]:
from sklearn.metrics import mean_absolute_error, r2_score
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

evaluate_model(model, x_test_vectorized, y_test)

(20.93596813650014, 0.886123315058664)

In [15]:
texto = "Zapatillas Adidas Running Ultra Boost 20 para niño"
df_inference = pd.DataFrame({"description": [texto]})

df_inference_processed = TextProcessingStep().execute(df_inference)

df_inference_vectorized = feature_extraction_step.execute(df_inference_processed, "description_processed", "inference")

model.predict(df_inference_vectorized)



array([128.44690901])